# nvImageCodec with TensorFlow

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
import torch

Set lower log level and import tensorflow

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

Import nvImageCodec

In [ ]:
from nvidia import nvimgcodec

Setting resource folder

In [ ]:
resources_dir = os.getenv("PYNVIMGCODEC_EXAMPLES_RESOURCES_DIR", "../assets/images/")

Created nvImageCodec Decoder

In [ ]:
decoder = nvimgcodec.Decoder()

Decode JPEG2000 file 

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")

Transfer image to Host memory and display using matplotlib.pyplot

In [ ]:
plt.imshow(nv_img.cpu())

Zero-copy convertion of Image to tensor using DLPack

In [ ]:
cap = nv_img.to_dlpack()
tf_img = tf.experimental.dlpack.from_dlpack(cap)
print("device:", tf_img.device)
print("dtype of tensor:", tf_img.dtype)

Lets flip tensor

In [ ]:
tf_img_flip = tf.image.flip_up_down(tf_img)

Pass tensor to cuPy and then to numpy to show flipped image

In [ ]:
cp_img = cp.from_dlpack(tf.experimental.dlpack.to_dlpack(tf_img_flip))
np_img = cp.asnumpy(cp_img)
plt.imshow(np_img)

Zero-copy convertion of  tensor back to Image  

In [ ]:
cap = tf.experimental.dlpack.to_dlpack(tf_img_flip)
nv_flipped_img = nvimgcodec.as_image(cap)

Save as Jpeg2000

In [ ]:
encoder = nvimgcodec.Encoder()
encoder.write("tf_flipped.j2k", nv_flipped_img)

Load with OpenCV to verify

In [ ]:
image = cv2.imread("tf_flipped.j2k")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)